In [1]:
import torch
import dqc

import pyscf

In [2]:
def DQC_HF(system, basis):
    atomzs, atomposs = dqc.parse_moldesc(system)
    atomposs
    mol = dqc.Mol((atomzs, atomposs), basis=basis)
    qc = dqc.HF(mol).run()
    ene = qc.energy()  # calculate the energy
    return ene

In [3]:
def PySCF_HF(system, basis):
    mol = pyscf.gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = pyscf.scf.HF(mol)
    ene = hf.kernel(verbose=0)
    return ene

In [18]:
from pyscf import cc

def PySCF_CCSDT(system, basis):
    mol = pyscf.gto.M(atom = system, basis = basis, unit="Bohr", verbose=0) # NB: PySCF uses Angstrom by default
    hf = pyscf.scf.HF(mol).run()
    cc = pyscf.cc.CCSD(hf)
    ene = cc.kernel()
    et = cc.ccsd_t()
    return cc.e_tot + et

In [12]:
def accuracy_comparison(system, basis):
    print("CCSD(T): ", PySCF_CCSDT(system, basis))
    print("HF, DQC: ", DQC_HF(system, basis))
    print("HF, PySCF: ", PySCF_HF(system, basis))

Hydrogen at equlibrium point:

In [19]:
accuracy_comparison("H 0 0 0; H 1.4 0 0", "6-31G")

CCSD(T):  -1.1516791660767662
HF, DQC:  tensor(-1.1267, dtype=torch.float64)
HF, PySCF:  -1.1267427044517944


In [20]:
%timeit DQC_HF("H 0 0 0; H 1.4 0 0", "6-31G")

15.9 ms ± 420 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%timeit PySCF_HF("H 0 0 0; H 1.4 0 0", "6-31G")

159 ms ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Nitrogen at equlibrium point:

In [22]:
accuracy_comparison("N 0 0 0; N 2.07 0 0", "6-31G")

CCSD(T):  -109.10264680005099
HF, DQC:  tensor(-108.8679, dtype=torch.float64)
HF, PySCF:  -108.8678750800519


In [23]:
%timeit DQC_HF("N 0 0 0; N 2.07 0 0", "6-31G")

44.3 ms ± 3.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
%timeit PySCF_HF("N 0 0 0; N 2.07 0 0", "6-31G")

157 ms ± 58.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Ammonia at equlibrium point:

In [25]:
accuracy_comparison("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

CCSD(T):  -56.29214916490412
HF, DQC:  tensor(-56.1610, dtype=torch.float64)
HF, PySCF:  -56.16102378396439


In [26]:
%timeit DQC_HF("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

48 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit PySCF_HF("N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721", "6-31G")

219 ms ± 29.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Benzene at equlibrium point:

In [28]:
benzene_coords = """C        0.598362921      0.000000000     -4.742986733;
                    C       -0.557705772     -0.354690359     -4.044822733;
                    C        1.754431614      0.354690359     -4.044822733;
                    H       -1.457130878     -0.630640582     -4.587995733;
                    H        2.653856720      0.630640582     -4.587995733;
                    C       -0.557705772     -0.354690359     -2.648492733;
                    C        1.754431614      0.354690359     -2.648492733;
                    H       -1.457130878     -0.630640582     -2.105319733;
                    H        2.653856720      0.630640582     -2.105319733;
                    C        0.598362921      0.000000000     -1.950328733;
                    H        0.598362921      0.000000000     -0.863981733;
                    H        0.598362921      0.000000000     -5.829333733"""

In [29]:
accuracy_comparison(benzene_coords, "6-31G")

CCSD(T):  -217.07797043621366
HF, DQC:  tensor(-216.7411, dtype=torch.float64)
HF, PySCF:  -216.74175971297797


In [30]:
%timeit DQC_HF(benzene_coords, "6-31G")

1.65 s ± 45.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit PySCF_HF(benzene_coords, "6-31G")

880 ms ± 55.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Naphtalene at equlibrium point:

In [81]:
naphtalene_coords = """H       -1.057571674      0.000000000     -4.361571149;
                    C       -1.057571674      0.487597890     -3.390034963;
                    C       -1.057571674      1.860916946     -3.299493774;
                    C       -1.057571674      2.493399263     -2.031832408;
                    C       -1.057571674      1.739878824     -0.880133713;
                    C       -1.057571674      0.320033769     -0.937967730;
                    C       -1.057571674     -0.320033769     -2.220831877;
                    C       -1.057571674     -1.739878824     -2.278665893;
                    C       -1.057571674     -2.493399263     -1.126967199;
                    C       -1.057571674     -1.860916946      0.140694168;
                    C       -1.057571674     -0.487597890      0.231235357;
                    H       -1.057571674      2.466184811     -4.201283252;
                    H       -1.057571674      3.577893185     -1.973126484;
                    H       -1.057571674      2.222891720      0.093690092;
                    H       -1.057571674      0.000000000      1.202771542;
                    H       -1.057571674     -2.466184811      1.042483645;
                    H       -1.057571674     -3.577893185     -1.185673122;
                    H       -1.057571674     -2.222891720     -3.252489699"""

In [82]:
accuracy_comparison(naphtalene_coords, "6-31G")

CCSD(T):  -360.2288580299286
HF, DQC:  tensor(-359.6521, dtype=torch.float64)
HF, PySCF:  -359.6605290827472


In [83]:
%timeit DQC_HF(naphtalene_coords, "6-31G")

12.7 s ± 1.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [84]:
%timeit PySCF_HF(naphtalene_coords, "6-31G")

4.31 s ± 1.18 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Anthracene at equlibrium point:

In [5]:
anthracene_coords = """C        1.044178831     -0.528233237     -4.684523771;
C        1.044178831     -0.552713285     -3.254782652;
C        1.044178831     -1.749044624     -2.526815306;
C        1.044178831      0.704131514     -2.541488804;
C        1.044178831      0.692578077     -1.141127549;
C        1.044178831     -0.503753261     -0.413160203;
C        1.044178831     -1.760598060     -1.126454051;
C        1.044178831      0.658108897     -5.368920348;
C        1.044178831      1.919058175     -3.295618865;
H        1.044178831      1.638563069     -0.604255156;
C        1.044178831     -0.528233309      1.016580916;
C        1.044178831     -2.975524721     -0.372323990;
C        1.044178831     -2.954793300      0.997119891;
C        1.044178831     -1.714574573      1.700977987;
H        1.044178831      0.418178836      1.551045394;
H        1.044178831     -3.919661467     -0.910797789;
H        1.044178831     -3.885538136      1.556731594;
H        1.044178831     -1.717772077      2.786998206;
C        1.044178831      1.898326754     -4.665062745;
H        1.044178831     -1.474645383     -5.218988249;
H        1.044178831     -2.695029615     -3.063687699;
H        1.044178831      0.661305531     -6.454941061;
H        1.044178831      2.863194921     -2.757145066;
H        1.044178831      2.829071589     -5.224674449"""

In [ ]:
%timeit DQC_HF(anthracene_coords, "6-31G")

In [ ]:
%timeit PySCF_HF(anthracene_coords, "6-31G")